## Project 2: Predicting Sale Price of Houses using Multivariate Linear Regression

* Student name: Andrea Cabello
* Full Time Online Student
* Instructor name: Rafael Carrasco

## I. Overview

We are presented with a file: *'kc_house_data.csv'*. This dataset contains info on King County House Sales (WA). We are tasked with the following:
* clean data (apply what we learned on Phase 1), 
* explore data (aka EDA) to obtaing meaningful insights, 
* model this dataset with a multivariate linear regression to predict the sale price of houses as accurately as possible.

## II. Business Problem


* What to look for when buying a home?
* What makes a residential property more expensive? (Location, Number of Rooms, Condition)
* Where are the most expensive houses located?
* Price per number of rooms (bedrooms/bathrooms)
* Price per sqft
* How does the condition of a property influence the price?
* How do year built and renovations influence the sale price?

## III. Understanding the Data
* Let's import the file and begin exploring.
* To understand what our features mean: https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r

In [192]:
import pandas as pd
import numpy as np

In [193]:
df = pd.read_csv('../data/kc_house_data.csv')

In [194]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


We'll begin by exploring one column at the time and find:
* what insight we could get from them,
* or how we could use them to engineer some useful features,
* which not to include them in the model

## Feature Variables
The factors we suspect have an impact on the price.

* ### Latitude & Longitude

Let's combine them in a tuple to obtain a location point.

In [195]:
[round(i,4) for i in df['lat']]
[round(i,4) for i in df['long']]

[-122.257,
 -122.319,
 -122.233,
 -122.393,
 -122.045,
 -122.005,
 -122.327,
 -122.315,
 -122.337,
 -122.031,
 -122.145,
 -122.292,
 -122.229,
 -122.045,
 -122.394,
 -122.375,
 -121.962,
 -122.343,
 -122.21,
 -122.306,
 -122.341,
 -122.233,
 -122.169,
 -122.166,
 -122.172,
 -122.218,
 -122.36,
 -122.314,
 -122.304,
 -122.11,
 -122.07,
 -122.357,
 -122.368,
 -122.375,
 -122.157,
 -122.31,
 -122.132,
 -122.362,
 -122.282,
 -122.18,
 -122.027,
 -122.347,
 -122.016,
 -122.364,
 -122.175,
 -121.977,
 -122.371,
 -122.151,
 -122.301,
 -122.451,
 -122.322,
 -122.189,
 -122.384,
 -122.369,
 -122.281,
 -122.29,
 -122.114,
 -122.122,
 -122.116,
 -122.149,
 -122.339,
 -122.335,
 -122.344,
 -122.32,
 -122.297,
 -122.304,
 -122.192,
 -122.257,
 -122.11,
 -122.215,
 -122.16,
 -122.179,
 -122.287,
 -122.036,
 -122.073,
 -121.987,
 -122.125,
 -122.394,
 -122.34,
 -122.025,
 -122.008,
 -122.031,
 -122.291,
 -122.149,
 -122.31,
 -122.365,
 -122.189,
 -122.199,
 -122.194,
 -122.387,
 -122.372,
 -122.391,


In [196]:
df['geo_location'] = tuple(zip(df['lat'], df['long']))

It sounds like a good idea to find the 'geo_location' of a point of reference like the center of the city, to calculate the distance from the properties to the point of reference. 
King County's main economic center is Seattle.

In [197]:
import sklearn.metrics as metrics
from math import sqrt
from haversine import haversine

In [198]:
# Defining Point of reference location:
Seattle = [47.6219, -122.3517]

Create lists to append the values we will obtain for distance between the location point of the property and the center point.

In [199]:
distance_from_seattle = []
for i in df['geo_location']:
    distance_from_seattle.append((haversine((Seattle),(i), unit='mi')))
rounded_distance_from_seattle = [round(i,2) for i in distance_from_seattle]

df['distance_seattle'] = pd.Series(rounded_distance_from_seattle)

In [200]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,geo_location,distance_seattle
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650,"(47.5112, -122.257)",8.83
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,"(47.721000000000004, -122.319)",7.01
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062,"(47.7379, -122.23299999999999)",9.73
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,"(47.5208, -122.39299999999999)",7.25
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503,"(47.6168, -122.045)",14.29


In [201]:
df.distance_seattle.isna().sum()

0

* ### Date
Since we have Columns expressed in 'years' time such as 'yr_built' and 'year renovated', first instinct is to look at the data by year.

In [202]:
df['yr_sold'] = df['date'].apply(lambda x: x[5:10])
df.yr_sold.unique()
# we have houses sold in 2014 and 2015.  

array(['/2014', '2014', '2015', '015', '014'], dtype=object)

In [203]:
df.yr_sold.value_counts()

2014     8659
2015     4784
/2014    3261
014      2702
015      2191
Name: yr_sold, dtype: int64

In [204]:
df.yr_sold = df.yr_sold.replace('/2014', '2014')
df.yr_sold = df.yr_sold.replace('014', '2014')
df.yr_sold = df.yr_sold.replace('015', '2015')
df.yr_sold = df.yr_sold.astype('int64')

In [205]:
df.yr_sold.value_counts()

2014    14622
2015     6975
Name: yr_sold, dtype: int64

* ### Year Built

In [206]:
df.yr_built.unique()

array([1955, 1951, 1933, 1965, 1987, 2001, 1995, 1963, 1960, 2003, 1942,
       1927, 1977, 1900, 1979, 1994, 1916, 1921, 1969, 1947, 1968, 1985,
       1941, 1915, 1909, 1948, 2005, 1929, 1981, 1930, 1904, 1996, 2000,
       1984, 2014, 1922, 1959, 1966, 1953, 1950, 2008, 1991, 1954, 1973,
       1925, 1989, 1972, 1986, 1956, 2002, 1992, 1964, 1952, 1961, 2006,
       1988, 1962, 1939, 1946, 1967, 1975, 1980, 1910, 1983, 1978, 1905,
       1971, 2010, 1945, 1924, 1990, 1914, 1926, 2004, 1923, 2007, 1976,
       1949, 1999, 1901, 1993, 1920, 1997, 1943, 1957, 1940, 1918, 1928,
       1974, 1911, 1936, 1937, 1982, 1908, 1931, 1998, 1913, 2013, 1907,
       1958, 2012, 1912, 2011, 1917, 1932, 1944, 1902, 2009, 1903, 1970,
       2015, 1934, 1938, 1919, 1906, 1935])

We'll use 'yr_sold' and 'yr_built' to create a feature **'property_age'** 

In [207]:
df['property_age'] = df['yr_sold'] - df['yr_built']

In [208]:
df.property_age.isnull().sum()

0

In [209]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                21597 non-null  int64  
 1   date              21597 non-null  object 
 2   price             21597 non-null  float64
 3   bedrooms          21597 non-null  int64  
 4   bathrooms         21597 non-null  float64
 5   sqft_living       21597 non-null  int64  
 6   sqft_lot          21597 non-null  int64  
 7   floors            21597 non-null  float64
 8   waterfront        19221 non-null  float64
 9   view              21534 non-null  float64
 10  condition         21597 non-null  int64  
 11  grade             21597 non-null  int64  
 12  sqft_above        21597 non-null  int64  
 13  sqft_basement     21597 non-null  object 
 14  yr_built          21597 non-null  int64  
 15  yr_renovated      17755 non-null  float64
 16  zipcode           21597 non-null  int64 

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,geo_location,distance_seattle,yr_sold,property_age
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,0.0,98178,47.5112,-122.257,1340,5650,"(47.5112, -122.257)",8.83,2014,59
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,1991.0,98125,47.7210,-122.319,1690,7639,"(47.721000000000004, -122.319)",7.01,2014,63
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,NaN,98028,47.7379,-122.233,2720,8062,"(47.7379, -122.23299999999999)",9.73,2015,82
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,0.0,98136,47.5208,-122.393,1360,5000,"(47.5208, -122.39299999999999)",7.25,2014,49
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,0.0,98074,47.6168,-122.045,1800,7503,"(47.6168, -122.045)",14.29,2015,28


* ### Waterfront

In [210]:
#dealing with nan values for waterfront feature
df.waterfront = df.waterfront.fillna('unknown')
df.waterfront = df.waterfront.replace(0.0, 0)
df.waterfront = df.waterfront.replace(1.0, 1)

In [211]:
df.waterfront.unique()

array(['unknown', 0, 1], dtype=object)

* ### Overall condition of the house

We have two columns with info on overall condition. After some additional googling I found that:
* **'condition'** summarizes grade and age in a scale from 1 to 5 where 1 = 'poor' and 5 = 'Very Good'
* **'grade'** represents the construction quality of improvements in a scale from 1 to 13 where 1-3 = 'Falls short of minimum building standards and  13 = 'Generally custom designed and built. Mansion level. Large amount of highest quality cabinet work, wood trim, marble, entry ways etc.'
Since this two are most likely multicollinear, I choose to work with 'condition' because is more scalable model wise.
Thinking about model scalability, I decide to work with *'condition'*

In [212]:
df.grade.unique()

array([ 7,  6,  8, 11,  9,  5, 10, 12,  4,  3, 13])

In [213]:
df.condition.unique()

array([3, 5, 4, 1, 2])

In [214]:
df.condition.value_counts()

3    14020
4     5677
5     1701
2      170
1       29
Name: condition, dtype: int64

* ### Year Renovated

In [215]:
df.yr_renovated.unique()

array([   0., 1991.,   nan, 2002., 2010., 1992., 2013., 1994., 1978.,
       2005., 2003., 1984., 1954., 2014., 2011., 1983., 1945., 1990.,
       1988., 1977., 1981., 1995., 2000., 1999., 1998., 1970., 1989.,
       2004., 1986., 2007., 1987., 2006., 1985., 2001., 1980., 1971.,
       1979., 1997., 1950., 1969., 1948., 2009., 2015., 1974., 2008.,
       1968., 2012., 1963., 1951., 1962., 1953., 1993., 1996., 1955.,
       1982., 1956., 1940., 1976., 1946., 1975., 1964., 1973., 1957.,
       1959., 1960., 1967., 1965., 1934., 1972., 1944., 1958.])

In [216]:
df.yr_renovated.isna().sum()

3842

* 78% of our data has a value of zero for year renovated, which we are assuming it means never renovated
* we only have 744 entries with a renovation year.
* maybe we should do a feature  'renovated': and break it down by values: yes, no and unknown

In [217]:
df['yr_renovated'] = df.yr_renovated.fillna('unknown')

In [218]:
to_replace = [1991., 2002., 2010., 1992., 2013., 1994., 1978.,
       2005., 2003., 1984., 1954., 2014., 2011., 1983., 1945., 1990.,
       1988., 1977., 1981., 1995., 2000., 1999., 1998., 1970., 1989.,
       2004., 1986., 2007., 1987., 2006., 1985., 2001., 1980., 1971.,
       1979., 1997., 1950., 1969., 1948., 2009., 2015., 1974., 2008.,
       1968., 2012., 1963., 1951., 1962., 1953., 1993., 1996., 1955.,
       1982., 1956., 1940., 1976., 1946., 1975., 1964., 1973., 1957.,
       1959., 1960., 1967., 1965., 1934., 1972., 1944., 1958.]

In [219]:
df['yr_renovated'] = df.yr_renovated.replace(to_replace=to_replace, value=1)

In [220]:
df['yr_renovated'] = df.yr_renovated.replace(0.0, 0)

In [221]:
df['renovation'] = df['yr_renovated'] 

In [222]:
df.renovation.value_counts()

0          17011
unknown     3842
1            744
Name: renovation, dtype: int64

* ### Bedrooms & Bathrooms

In [223]:
df.bedrooms.unique()

array([ 3,  2,  4,  5,  1,  6,  7,  8,  9, 11, 10, 33])

In [224]:
df.drop(df.loc[df['bedrooms']==33].index, inplace=True)

To better understand the way bathrooms are quantified, I went to Google and typed: 'what does .25 bathroom mean'.
I found out that:  "what Realtors now call a Full Bathroom (1) consists of a sink, toilet, bathtub and shower.
Because each of those items counts as .25, the convention is to call a Bathroom missing one or more a fractional Bath" [source](https://rosskaplan.com/2018/02/the-case-for-the-1-25-bath-or-the-1-375-bath/#:~:text=At%20least%2C%20that's%20what%20Realtors,is%20a%203%2F4%20Bath.) .

In [225]:
df.bathrooms.unique()

array([1.  , 2.25, 3.  , 2.  , 4.5 , 1.5 , 2.5 , 1.75, 2.75, 3.25, 4.  ,
       3.5 , 0.75, 4.75, 5.  , 4.25, 3.75, 1.25, 5.25, 6.  , 0.5 , 5.5 ,
       6.75, 5.75, 8.  , 7.5 , 7.75, 6.25, 6.5 ])

* ### Floors

In [226]:
df.floors.unique()

array([1. , 2. , 1.5, 3. , 2.5, 3.5])

In [227]:
df.floors.value_counts()

1.0    10672
2.0     8235
1.5     1910
3.0      611
2.5      161
3.5        7
Name: floors, dtype: int64

We'll make a new column **'rooms_per_floor'** using ('bedrooms' + 'bathrooms') / 'floors'.

In [228]:
df['rooms_per_floor'] = (df['bedrooms'] + df['bathrooms']) / df['floors']

In [229]:
df['rooms_per_floor'] = [round(i,2) for i in df.rooms_per_floor]

In [230]:
df.rooms_per_floor.value_counts()

3.25     2094
4.00     1855
2.75     1806
3.00     1690
4.75     1632
         ... 
5.60        1
1.29        1
3.70        1
3.43        1
11.75       1
Name: rooms_per_floor, Length: 130, dtype: int64

In [231]:
df.rooms_per_floor.describe()

count    21596.000000
mean         4.003967
std          1.416580
min          0.750000
25%          3.000000
50%          3.750000
75%          4.750000
max         15.250000
Name: rooms_per_floor, dtype: float64

* ### sqft_living,	sqft_lot, sqft_above, sqft_basement

These columns are  measurements of the area of a property expressed in square feet. Compared to 'rooms_per_floor', it's just another way to measure the living area but in different units.  These will most likely have multicollinearity with the number of rooms per floor so I didn't inlcude them in my first attempt.

In [232]:
ninety_perc0 = df['sqft_lot'].quantile(0.9) # get the 90th percentile

In [233]:
df_90 = df.loc[df['sqft_lot']<=ninety_perc0]

In [234]:
df_90.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19436 entries, 0 to 21596
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                19436 non-null  int64  
 1   date              19436 non-null  object 
 2   price             19436 non-null  float64
 3   bedrooms          19436 non-null  int64  
 4   bathrooms         19436 non-null  float64
 5   sqft_living       19436 non-null  int64  
 6   sqft_lot          19436 non-null  int64  
 7   floors            19436 non-null  float64
 8   waterfront        19436 non-null  object 
 9   view              19378 non-null  float64
 10  condition         19436 non-null  int64  
 11  grade             19436 non-null  int64  
 12  sqft_above        19436 non-null  int64  
 13  sqft_basement     19436 non-null  object 
 14  yr_built          19436 non-null  int64  
 15  yr_renovated      19436 non-null  object 
 16  zipcode           19436 non-null  int64 

In [235]:
df_90.sqft_lot.describe()

count    19436.000000
mean      7556.162894
std       3888.764310
min        520.000000
25%       4950.000000
50%       7212.500000
75%       9570.250000
max      21370.000000
Name: sqft_lot, dtype: float64

## Target Variable
### Price
This is our dependent variable or the one we want to predict.

In [236]:
df_90.price.describe()

count    1.943600e+04
mean     5.219898e+05
std      3.342399e+05
min      7.800000e+04
25%      3.150000e+05
50%      4.400000e+05
75%      6.250000e+05
max      4.670000e+06
Name: price, dtype: float64

In [237]:
ninety_perc1 = df_90['price'].quantile(0.9) # get the 90th percentile

In [238]:
df_90 = df_90.loc[df_90['price']<=ninety_perc1]

In [239]:
df_90.price.describe()

count     17492.000000
mean     438940.835296
std      171256.438049
min       78000.000000
25%      302000.000000
50%      415000.000000
75%      555000.000000
max      856600.000000
Name: price, dtype: float64

In [240]:
df_90.sqft_lot.describe()

count    17492.000000
mean      7345.552767
std       3736.009050
min        520.000000
25%       4835.250000
50%       7200.000000
75%       9282.750000
max      21370.000000
Name: sqft_lot, dtype: float64

In [241]:
df_90.distance_seattle.describe()

count    17492.000000
mean        11.842841
std          6.957736
min          0.330000
25%          6.050000
50%         10.500000
75%         16.500000
max         48.650000
Name: distance_seattle, dtype: float64

In [242]:
ninety_perc2 = df_90['distance_seattle'].quantile(0.9)

In [243]:
df_90 = df_90.loc[df_90['distance_seattle']<=ninety_perc2]

In [244]:
df_90.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,view,condition,grade,...,yr_built,zipcode,lat,long,sqft_living15,sqft_lot15,distance_seattle,yr_sold,property_age,rooms_per_floor
count,1.574200e+04,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000,15696.000000,15742.000000,15742.000000,...,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000
mean,4.703359e+09,450911.570576,3.274298,1.972145,1840.128827,7254.739169,1.444321,0.136468,3.411638,7.397599,...,1968.176534,98086.272329,47.578413,-122.244937,1813.543641,7232.731292,10.363642,2014.321624,46.145090,3.985293
std,2.873330e+09,171196.157571,0.890540,0.686504,672.382422,3752.395860,0.544765,0.559416,0.650685,0.918523,...,29.506388,54.053872,0.126039,0.114092,531.915026,4529.591680,5.518193,0.467114,29.510392,1.427181
min,1.000102e+06,78000.000000,1.000000,0.500000,370.000000,520.000000,1.000000,0.000000,1.000000,4.000000,...,1900.000000,98001.000000,47.300100,-122.512000,460.000000,651.000000,0.330000,2014.000000,-1.000000,0.750000
25%,2.249575e+09,315000.000000,3.000000,1.500000,1340.000000,4750.000000,1.000000,0.000000,3.000000,7.000000,...,1949.000000,98034.000000,47.493600,-122.341000,1430.000000,4820.000000,5.610000,2014.000000,21.000000,3.000000
50%,4.028301e+09,428000.000000,3.000000,2.000000,1750.000000,7168.500000,1.000000,0.000000,3.000000,7.000000,...,1968.000000,98103.000000,47.584800,-122.280000,1720.000000,7200.500000,9.805000,2014.000000,46.000000,3.750000
75%,7.408700e+09,569000.000000,4.000000,2.500000,2256.000000,9219.750000,2.000000,0.000000,4.000000,8.000000,...,1994.000000,98125.000000,47.687175,-122.166000,2120.000000,8904.750000,14.380000,2015.000000,66.000000,4.750000
max,9.900000e+09,856600.000000,11.000000,7.500000,5820.000000,21370.000000,3.500000,4.000000,5.000000,11.000000,...,2015.000000,98199.000000,47.777600,-121.886000,4362.000000,196591.000000,22.250000,2015.000000,115.000000,11.500000


In [245]:
features_target = ['distance_seattle', 'property_age', 
                   'renovation', 'waterfront', 'rooms_per_floor', 
                   'sqft_lot', 'sqft_living', 'price']

In [246]:
df1 = df_90[features_target]
df1 = pd.get_dummies(df1, drop_first=True)

In [247]:
df1.info()
df1.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15742 entries, 0 to 21596
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   distance_seattle    15742 non-null  float64
 1   property_age        15742 non-null  int64  
 2   rooms_per_floor     15742 non-null  float64
 3   sqft_lot            15742 non-null  int64  
 4   sqft_living         15742 non-null  int64  
 5   price               15742 non-null  float64
 6   renovation_1        15742 non-null  uint8  
 7   renovation_unknown  15742 non-null  uint8  
 8   waterfront_1        15742 non-null  uint8  
 9   waterfront_unknown  15742 non-null  uint8  
dtypes: float64(3), int64(3), uint8(4)
memory usage: 922.4 KB


,distance_seattle,property_age,rooms_per_floor,sqft_lot,sqft_living,price,renovation_1,renovation_unknown,waterfront_1,waterfront_unknown
0,8.83,59,4.00,5650,1180,221900.0,0,0,0,1
1,7.01,63,2.62,7242,2570,538000.0,1,0,0,0
2,9.73,82,3.00,10000,770,180000.0,0,1,0,0
3,7.25,49,7.00,5000,1960,604000.0,0,0,0,0
4,14.29,28,5.00,8080,1680,510000.0,0,0,0,0


In [191]:
df1.describe()

,distance_seattle,property_age,rooms_per_floor,sqft_lot,sqft_living,price,renovation_1,renovation_unknown,waterfront_1,waterfront_unknown
count,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000,15742.000000
mean,10.363642,46.145090,3.985293,7254.739169,1840.128827,450911.570576,0.027315,0.179456,0.001461,0.111485
std,5.518193,29.510392,1.427181,3752.395860,672.382422,171196.157571,0.163006,0.383746,0.038197,0.314742
min,0.330000,-1.000000,0.750000,520.000000,370.000000,78000.000000,0.000000,0.000000,0.000000,0.000000
25%,5.610000,21.000000,3.000000,4750.000000,1340.000000,315000.000000,0.000000,0.000000,0.000000,0.000000
50%,9.805000,46.000000,3.750000,7168.500000,1750.000000,428000.000000,0.000000,0.000000,0.000000,0.000000
75%,14.380000,66.000000,4.750000,9219.750000,2256.000000,569000.000000,0.000000,0.000000,0.000000,0.000000
max,22.250000,115.000000,11.500000,21370.000000,5820.000000,856600.000000,1.000000,1.000000,1.000000,1.000000


In [190]:
%store df1

Stored 'df1' (DataFrame)
